In [2]:
!pip install bayesian-optimization
import pickle

import numpy as np
import pandas as pd
import os.path as osp
import os
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler, OneHotEncoder
)
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis #LDA
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel #para seleccionar los coeficientes de lasso
from sklearn.metrics import roc_auc_score, f1_score
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
set_config(display="diagram")

from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
DATA_DIR = "/content/drive/MyDrive/2024/05_maestria/semestre2/fundamentos2/taller1_javier"
df = pd.read_parquet(osp.join(DATA_DIR, "df_train.parquet"))
print(df.shape)
df.head()

(23494, 46)


,MEDICAMENTOS,MEDICINA ESPECIALIZADA,MEDICINA GENERAL,Cant_gr_flia,Cant_riesgos_flia_mean,min_Tiempo_CP_Fliar,Cant_Fliar_CP,psa_max_gr_flia,psa_min_gr_flia,Cant_Fliar_riesgos,...,CORONARIOS_FAMILIAR,CEREBRAL,CEREBRAL_FAMILIAR,ENFERMEDAD_RENAL,ENFERMEDAD_RENAL_FAMILIAR,OTROS_ANTECEDENTES_VASCULARES,Target,Pendiente,Intercepto,Promedio_costo
ID,,,,,,,,,,,,,,,,,,,,,
16484,12.0,3.0,3.0,2,0.0,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,58373.057143,-110021.200000,1.885690e+05
11528,6.0,1.0,1.0,2,0.5,NaN,0.0,NaN,NaN,1.0,...,0,0,0,0,0,0,0,19618.942857,-36176.133333,9.747050e+04
26630,1.0,0.0,2.0,2,0.0,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,1,398534.000000,-626628.000000,2.304723e+06
12586,6.0,0.0,2.0,2,0.0,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,1292.600000,13410.733333,5.380450e+04
11225,3.0,0.0,2.0,1,1.0,NaN,0.0,NaN,NaN,1.0,...,2,2,2,2,2,2,0,13728.714286,-27375.666667,4.134967e+04


In [5]:
df1 = pd.read_parquet(osp.join(DATA_DIR, "df_test.parquet"))
print(df1.shape)
df1.head()

(10069, 45)


,MEDICAMENTOS,MEDICINA ESPECIALIZADA,MEDICINA GENERAL,Cant_gr_flia,Cant_riesgos_flia_mean,min_Tiempo_CP_Fliar,Cant_Fliar_CP,psa_max_gr_flia,psa_min_gr_flia,Cant_Fliar_riesgos,...,CORONARIOS,CORONARIOS_FAMILIAR,CEREBRAL,CEREBRAL_FAMILIAR,ENFERMEDAD_RENAL,ENFERMEDAD_RENAL_FAMILIAR,OTROS_ANTECEDENTES_VASCULARES,Pendiente,Intercepto,Promedio_costo
ID,,,,,,,,,,,,,,,,,,,,,
12596,2.0,4.0,2.0,2,0.000000,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,5.421180e+06,-1.256617e+07,9.611937e+06
29326,4.0,0.0,1.0,1,0.000000,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,-9.504343e+03,5.997787e+04,8.013800e+04
24338,0.0,0.0,1.0,2,0.000000,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,2.477143e+03,-5.780000e+03,1.734000e+04
11167,1.0,0.0,0.0,1,0.000000,NaN,0.0,NaN,NaN,0.0,...,1,0,1,0,1,0,0,3.597143e+03,-8.393333e+03,2.518000e+04
22074,36.0,0.0,1.0,3,0.666667,NaN,0.0,NaN,NaN,1.0,...,0,0,0,0,0,0,0,-9.042514e+03,6.309881e+05,5.993393e+05


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23494 entries, 16484 to 9409
Data columns (total 46 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   MEDICAMENTOS                   23494 non-null  float64
 1   MEDICINA ESPECIALIZADA         23494 non-null  float64
 2   MEDICINA GENERAL               23494 non-null  float64
 3   Cant_gr_flia                   23494 non-null  int64  
 4   Cant_riesgos_flia_mean         23494 non-null  float64
 5   min_Tiempo_CP_Fliar            8 non-null      float64
 6   Cant_Fliar_CP                  23494 non-null  float64
 7   psa_max_gr_flia                164 non-null    float64
 8   psa_min_gr_flia                164 non-null    float64
 9   Cant_Fliar_riesgos             23494 non-null  float64
 10  cantidad_serv_flia             23494 non-null  float64
 11  Pendiente_flia                 23494 non-null  float64
 12  Intercepto_flia                23494 non-null  f

In [8]:
X, y = df.drop(columns="Target"), df["Target"]
y.value_counts(True) * 100

,proportion
Target,
0,71.537414
1,28.462586


In [11]:
porcetaje_de_nulidad = (
    X.isnull()
    .apply(lambda s: s.value_counts(True)).T
)

porcetaje_de_nulidad.columns = ['not_null', 'null']
variables_muy_nulas = porcetaje_de_nulidad.query('null > 0.7').index
variables_muy_nulas

Index(['min_Tiempo_CP_Fliar', 'psa_max_gr_flia', 'psa_min_gr_flia',
       'PERDIDA_DE_PESO'],
      dtype='object')

In [13]:
X = X.drop(columns=variables_muy_nulas)
X.head()

,MEDICAMENTOS,MEDICINA ESPECIALIZADA,MEDICINA GENERAL,Cant_gr_flia,Cant_riesgos_flia_mean,Cant_Fliar_CP,Cant_Fliar_riesgos,cantidad_serv_flia,Pendiente_flia,Intercepto_flia,...,CORONARIOS,CORONARIOS_FAMILIAR,CEREBRAL,CEREBRAL_FAMILIAR,ENFERMEDAD_RENAL,ENFERMEDAD_RENAL_FAMILIAR,OTROS_ANTECEDENTES_VASCULARES,Pendiente,Intercepto,Promedio_costo
ID,,,,,,,,,,,,,,,,,,,,,
16484,12.0,3.0,3.0,2,0.0,0.0,0.0,34.0,15998.485714,-23363.200000,...,0,0,0,0,0,0,0,58373.057143,-110021.200000,1.885690e+05
11528,6.0,1.0,1.0,2,0.5,0.0,1.0,25.0,655.200000,-1019.200000,...,0,0,0,0,0,0,0,19618.942857,-36176.133333,9.747050e+04
26630,1.0,0.0,2.0,2,0.0,0.0,0.0,36.0,342236.971429,-521835.400000,...,0,0,0,0,0,0,0,398534.000000,-626628.000000,2.304723e+06
12586,6.0,0.0,2.0,2,0.0,0.0,0.0,24.0,-4468.285714,30687.333333,...,0,0,0,0,0,0,0,1292.600000,13410.733333,5.380450e+04
11225,3.0,0.0,2.0,1,1.0,0.0,1.0,7.0,0.000000,0.000000,...,2,2,2,2,2,2,2,13728.714286,-27375.666667,4.134967e+04


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [15]:
var_numericas = X_train.select_dtypes('number').columns
var_categoricas = X_train.select_dtypes('object').columns

In [16]:
print(y_train.value_counts(True)*100)
print(y_test.value_counts(True)*100)

Target
0    71.590966
1    28.409034
Name: proportion, dtype: float64
Target
0    71.234043
1    28.765957
Name: proportion, dtype: float64


# Data Processing

Ejemplo sobre como usar Pipelines: https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html

https://towardsdatascience.com/creating-custom-transformers-for-sklearn-pipelines-d3d51852ecc1

In [17]:
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
import numpy as np

# Selección de las variables numéricas y categóricas
numeric_cols = X_train.select_dtypes('number').columns
categorical_cols = X_train.select_dtypes('object').columns

# Transformer para las variables categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=np.nan)),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

# Transformer para las variables numéricas, incluyendo la selección de características
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=3, random_state=42))  # Aplica PCA a las variables numéricas seleccionadas
])

# Preprocesador que maneja tanto numéricas como categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ], remainder='passthrough'
)

# Selección de características con Lasso
feature_selector = SelectFromModel(LassoCV(cv=5, random_state=42))

# Pipeline que ahora incluye la selección de características con Lasso y un clasificador XGBoost
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selector', feature_selector),  # Selección de características con Lasso
    ('classifier', XGBClassifier(random_state=42))
])

In [19]:
from sklearn.model_selection import cross_val_score

from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

# Definir la función de evaluación para XGBoost
def xgb_evaluate(n_estimators, max_depth, learning_rate, min_child_weight, base_score, n_components):

    # Actualiza los hiperparámetros del clasificador XGBoost
    pipeline.set_params(classifier__n_estimators=int(n_estimators),
                        classifier__max_depth=int(max_depth),
                        classifier__learning_rate=learning_rate,
                        classifier__min_child_weight=int(min_child_weight),
                        classifier__base_score=base_score)

    # Actualiza el número de componentes de PCA
    pipeline.set_params(preprocessor__num__pca__n_components=int(n_components))

    # Realiza validación cruzada y retorna la métrica (roc_auc)
    scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring='roc_auc')

    # Regresa el score promedio
    return scores.mean()

# Espacio de búsqueda de hiperparámetros, incluyendo n_components para PCA
param_bounds = {
    'n_estimators': (100, 1500),
    'max_depth': (2, 15),
    'learning_rate': (0.05, 0.95),
    'min_child_weight': (1, 4),
    'base_score': (0.25, 1.0),
    'n_components': (2, 10)  # Número de componentes para PCA
}

# Crear optimizador bayesiano
optimizer = BayesianOptimization(
    f=xgb_evaluate,
    pbounds=param_bounds,
    random_state=42,
    verbose=2
)

In [20]:
# Realizar la optimización bayesiana
optimizer.maximize(init_points=5, n_iter=10)

# Imprimir la mejor combinación de hiperparámetros y el mejor score
print(f"Best parameters: {optimizer.max['params']}")
print(f"Best score: {optimizer.max['target']}")

|   iter    |  target   | base_s... | learni... | max_depth | min_ch... | n_comp... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.7311    | 0.5309    | 0.9056    | 11.52     | 2.796     | 3.248     | 318.4     |
| 2         | 0.7164    | 0.2936    | 0.8296    | 9.814     | 3.124     | 2.165     | 1.458e+03 |
| 3         | 0.7019    | 0.8743    | 0.2411    | 4.364     | 1.55      | 4.434     | 834.7     |
| 4         | 0.7402    | 0.574     | 0.3121    | 9.954     | 1.418     | 4.337     | 612.9     |
| 5         | 0.6865    | 0.5921    | 0.7567    | 4.596     | 2.543     | 6.739     | 165.0     |
| 6         | 0.7411    | 0.4873    | 0.5359    | 9.95      | 1.732     | 4.998     | 612.1     |
| 7         | 0.7573    | 0.8602    | 0.3806    | 7.034     | 4.0       | 10.0      | 571.9     |
| 8         | 0.7582    | 0.25      | 0.95      | 15.0      | 4.0       | 10.0      | 499.9     |
| 9         | 0.5828

ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 473, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/usr/local/lib/python3.10/dist-packages/xgboost/core.py", line 726, in inner_f
    return func(**kwargs)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py", line 1531, 

In [25]:
best_params = optimizer.max['params']

best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['max_depth'] = int(best_params['max_depth'])
best_params['min_child_weight'] = int(best_params['min_child_weight'])
best_params['n_components'] = int(best_params['n_components'])

pipeline.set_params(classifier__n_estimators=best_params['n_estimators'],
                    classifier__max_depth=best_params['max_depth'],
                    classifier__learning_rate=best_params['learning_rate'],
                    classifier__min_child_weight=best_params['min_child_weight'],
                    classifier__base_score=best_params['base_score'],
                    preprocessor__num__pca__n_components=best_params['n_components'])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA(n_components=10,
                                                                       random_state=42))]),
                                                  Index(['MEDICAMENTOS', 'MEDICINA ESPECIALIZADA', 'MEDICINA GENERAL',
       'Cant_gr_flia', 'Cant_riesgos_flia_mean', 'Cant_Fliar_CP',
       'Cant_Fliar_ries...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.95,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=15, max_leaves=None,
                               min_child_weight=4, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=499, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [26]:
y_pred = pipeline.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred)
print(f'AUC para MLP: {auc}')

AUC para MLP: 0.8332897381698043


# Submission

In [27]:
test_df = pd.read_parquet(osp.join(DATA_DIR, "df_test.parquet"))
submission_pred = pipeline.predict_proba(test_df)[:, 1]
submission = pd.DataFrame(data=dict(ID=test_df.index, Target=submission_pred))

In [30]:
submission.to_csv(osp.join(DATA_DIR, "submission_Ome_Vega.csv"), index=False)